# Create a SDS file structure by retrieving data from a FDSN web server

If you need a SDS file structure, but only have a FDSN web server to provide data, then you can run a series of data requests to te server and create SDS.

For GeoNet, we have the particular issue of two FDSN web servers. You can get around this by retrieving from each server and merging the streams, taking care to check that the streams exist.

In [ ]:
from obspy.core import UTCDateTime
from obspy.clients.fdsn import Client
import pandas as pd
import os

In [ ]:
#GeoNet's FDSN web servers
arc_client = 'http://service.geonet.org.nz'
nrt_client = 'http://service-nrt.geonet.org.nz'

In [ ]:
#SDS file structure to create
sds = '/home/sherburn/SDS/geonet_sds'

In [ ]:
#GeoNet station list
stations = ['KUTZ', 'WPRZ', 'ALRZ', 'ARAZ', 'HRRZ', 'PRRZ', 'THQ2']
# stations = ['KUTZ']
net = 'NZ'
loc = '10'
components = ['EHZ', 'EHN', 'EHE']
# components = ['EHE']

In [ ]:
#get data for one day, try both servers
def daywave(date, net, sta, loc, cmp):
    global stnrt
    global starc
    
    time = UTCDateTime(date)
        
    #arc client
    try:
        client = Client(arc_client)
        starc = client.get_waveforms(net, sta, loc, cmp, time, time + 86400)
        print ('arc client successful')
    except:
        print ('arc client not successful')
                
    #nrt client
    try:
        client = Client(nrt_client)
        stnrt = client.get_waveforms(net, sta, loc, cmp, time, time + 86400)
        print ('nrt client successful')
    except:
        print ('nrt client not successful')   

In [ ]:
#date range for waveform data, dates are inclusive
#edit next two lines to set date range
date1 = '20190401'
date2 = '20190430'

In [ ]:
for sta in stations:
    for cmp in components:
        for date in pd.date_range(start=date1, end=date2, freq='D'):
            print (sta, cmp, loc, net, date)
            year = date.timetuple().tm_year
            yday = date.timetuple().tm_yday
            folder = os.path.join(sds, str(year), net, sta, cmp+'.D')
            if not os.path.exists(folder):
                os.makedirs(folder)
            file = os.path.join(folder, net+'.'+sta+'.'+loc+'.'+cmp+'.D.'+str(year)+'.'+str(yday).zfill(3))
            print (file)
            
            daywave(date, net, sta, loc, cmp)
            
            #check starc
            try:
                starc
                st = starc
                del starc
                print ('st = starc')
                try:
                    stnrt #if defined, add stnrt
                    st += stnrt
                    del stnrt
                    print ('st = starc+stnrt')
                except NameError: print ('stnrt not defined')
            except NameError: print ('starc not defined')
                
            #check stnrt, only passes if not already added to st
            try:
                stnrt
                st = stnrt
                del stnrt
                print ('st = stnrt')
            except NameError: print ('stnrt not defined')
            
            st.merge(fill_value = 'interpolate')
            print (st)
            st.write(file, format="MSEED")
            print ('---------')
           